In [1]:
import numpy as np
import networkx as nx
from networkx.algorithms.dag import is_directed_acyclic_graph
from multiprocessing import Pool
import pandas as pd
import random

from typing import List

from simulated import Simulated
from simulatedTimeSeries import SimulatedTimeSeries



In [2]:
n_series = 10  # You can change this as needed
n_observations = 10
n_variables = 3
# Testing with a single process
generator = SimulatedTimeSeries(n_series, n_observations, n_variables, maxlags=1)
generator.generate()
dags = generator.get_dags()
DAG = generator.get_updated_dags()
data = generator.get_observations()



current lag: 1
      0     1     2
0  0.02  0.97  0.83
current lag: 1
      0     1     2
0  0.14  0.29  0.37
current lag: 1
      0     1    2
0  0.97  0.81  0.3
current lag: 1
      0     1     2
0  0.66  0.31  0.52
current lag: 1
      0     1     2
0  0.83  0.36  0.28
current lag: 1
      0     1     2
0  0.82  0.71  0.73
current lag: 1
      0     1     2
0  0.89  0.47  0.12
current lag: 1
      0     1     2
0  0.11  0.03  0.64
current lag: 1
      0     1     2
0  0.93  0.81  0.63
current lag: 1
      0     1     2
0  0.11  0.23  0.43


In [4]:
def plot(DAG):
    from graphviz import Digraph

    G_dot = Digraph(engine="dot",format='png')

    for node in DAG.nodes():
        G_dot.node(str(node))
    for edge in DAG.edges():
        G_dot.edge(str(edge[0]), str(edge[1]))

    # Render the graph in a hierarchical layout
    #save the graph
    G_dot.render('graph', view=True)


In [5]:
import networkx as nx

def print_dag_details(DAG):
    # Print all nodes with their attributes
    print("Nodes and their attributes:")
    for node, attributes in DAG.nodes(data=True):
        print(node, attributes)
    
    # Print a separator for clarity
    print("\n" + "-"*50 + "\n")
    
    # Print all edges with their attributes
    print("Edges and their attributes:")
    for source, target, attributes in DAG.edges(data=True):
        print(f"({source}, {target})", attributes)

# Example usage:




In [6]:
print_dag_details(DAG[0])
plot(DAG[0])

Nodes and their attributes:
1 {'bias': 0.5, 'sigma': 0.1, 'seed': 42, 'value': -3.83}
0 {'bias': -0.14, 'sigma': 0.1, 'seed': 42, 'value': 3.5999999999999996}
2 {'bias': 0.65, 'sigma': 0.1, 'seed': 42, 'value': 2.31}
2_t-1 {'bias': 0.65, 'sigma': 0.1, 'seed': 42, 'value': 2.28}
1_t-1 {'bias': 0.5, 'sigma': 0.1, 'seed': 42, 'value': -3.7199999999999998}
0_t-1 {'bias': -0.14, 'sigma': 0.1, 'seed': 42, 'value': 3.18}

--------------------------------------------------

Edges and their attributes:
(1, 0) {'weight': -1.0, 'H': 'linear'}
(2, 0) {'weight': -1.0, 'H': 'linear'}
(2, 1) {'weight': -1.0, 'H': 'linear'}
(2_t-1, 2) {'weight': 0.73, 'H': 'linear'}
(2_t-1, 0) {'weight': -1.0, 'H': 'linear'}
(2_t-1, 1) {'weight': -1.0, 'H': 'linear'}
(1_t-1, 1) {'weight': 0.2, 'H': 'linear'}
(1_t-1, 0) {'weight': -1.0, 'H': 'linear'}
(0_t-1, 0) {'weight': 0.42, 'H': 'linear'}


In [20]:
first_line = data[1].iloc[1,:]
first_line

0   -7.60
1    4.44
2   -0.58
Name: 1, dtype: float64

In [18]:

dag_dict = {}
for node in nx.topological_sort(DAG[0]):
    if f"_t-" in str(node):
        idx = int(str(node)[0])
        dag_dict[node] = first_line[idx]
    else:
        dag_dict[node] = 0
        parents = list(DAG[0].predecessors(node))
        for parent in parents:
            dag_dict[node] += dag_dict[parent] * DAG[0].edges[parent, node]['weight'] + DAG[0].nodes[node]['bias']


In [19]:
dag_dict

{'2_t-1': 4.4399999999999995,
 '1_t-1': 4.4399999999999995,
 '0_t-1': 4.4399999999999995,
 2: 3.8911999999999995,
 1: -5.943199999999999,
 0: -5.663199999999998}

,0,1,2
0,0.02,0.97,0.83
1,-3.35,-0.40,1.26
2,-3.13,-1.41,1.57
3,-1.82,-2.15,1.80
4,-0.38,-2.69,1.96
5,0.87,-3.08,2.08
6,1.87,-3.37,2.17
7,2.63,-3.57,2.23
8,3.18,-3.72,2.28
9,3.60,-3.83,2.31


In [8]:
list(nx.topological_sort(DAG[0]))

['2_t-1', '1_t-1', '0_t-1', 2, 1, 0]

In [13]:
0.5+1.26*(-1)+0.83*(-1)+0.97*(0.2)

-1.396